# Blind Spot Map

Run and check at http://127.0.0.1:8050/

### Data Preparation

In [1]:
import pandas as pd
import numpy as np

#df = pd.read_excel("https://raw.githubusercontent.com/laurens-kolenbrander/DEiA-2/blob/73ac2e4abc9fda81321891ed8de4da258670c27c/2019_cbs_data.xls?token=GHSAT0AAAAAABUAYSV2GPBNRDKI4XE6TY2UYTPOCYA")
df = pd.read_excel("2019_cbs_data.xls")

regio_type = ['Land','Gemeente', 'Wijk']
df = df[~df.recs.isin(regio_type)]
df.head()

# If the files are already downloaded, set from_scratch = False
from_scratch = False

In [2]:
def minimal_distance(distances = pd.read_csv("neighbourhood_distances.csv")):
    ''' Calculate minimum distance for each facility type in each neighbourhood
    '''
    minimals = pd.DataFrame([],columns=['neighbourhood', 'sport_distance', 'play_distance','park_distance'])
    short = distances[['neighbourhood', 'distance', 'category']]
    
    sport = short[short['category']=='sport']
    park = short[short['category']=='park']
    playground = short[short['category']=='playground']
    neighbourhoods = short['neighbourhood']
    filt = neighbourhoods.drop_duplicates()

    for i in filt:
        single = sport[sport['neighbourhood']==i]
        single = single.sort_values(by=['distance'])
        sportmin = single.iloc[0]['distance']

        psingle = park[park['neighbourhood']==i]
        psingle = psingle.sort_values(by=['distance'])
        parkmin = psingle.iloc[0]['distance']

        plsingle = playground[playground['neighbourhood']==i]
        plsingle = plsingle.sort_values(by=['distance'])
        playmin = plsingle.iloc[0]['distance']
        
        minimals = pd.concat([minimals, pd.DataFrame([[i, sportmin, playmin, parkmin]],columns=['neighbourhood', 'sport_distance', 'play_distance','park_distance'])])
        
    return minimals

if from_scratch:
    # Import of neighbourhood distances
    distances = pd.read_csv("neighbourhood_distances.csv")

    minimals = minimal_distance()
    # Save to csv
    minimals.to_csv('minimaldistances_concat.csv')
    
else:
    # Importing minimal distance
    minimals = pd.read_csv('minimaldistances_concat.csv')
    minimals = minimals.drop(columns='Unnamed: 0')

In [3]:
# Translation of column names of the dataset

df_filtered = df[['regio', # neighbourhood
                  'gm_naam', # municipality, only for simple reading afterwards
                  'a_inw', #inhabitants
                  'a_vrouw', #women amount
                  'a_00_14', #0-15
                  'a_15_24', #15-24
                  'a_25_44', #25-44
                  'a_45_64', #45-64
                  'a_geb', #births
                  'a_hh_m_k', # households with children
                  'a_ongeh', #unmarried
                  'p_huurw',  # perc. rental 
                  'p_arb_pp', # perc. nett working participaton
                  'a_opl_lg',  # amount low education
                  'a_opl_md',  #amount mid edu
                  'a_opl_hg', # amount high edu
                  'g_hh_sti', # average standardized income
                  'g_pau_hh', #cars per hh
                  'pst_mvp' # most occuring postal
                 ]]


df_renamed = df_filtered.rename(columns={'regio': 'neighbourhood', 
                                         'gm_naam': 'municipality',
                                         'a_inw': 'citizens', 
                                         'a_vrouw': 'females', 
                                         'a_ongeh': 'not_married', 
                                         'p_huurw': 'rental_perc', 
                                         'p_arb_pp': 'percentage_working', 
                                         'a_opl_lg': 'low_edu',  
                                         'a_opl_md': 'mid_edu',  
                                         'a_opl_hg': 'high_edu', 
                                         'a_geb': 'births',
                                         'g_hh_sti': 'avg_income',
                                         'a_hh_m_k': 'households_with_children', 
                                         'g_pau_hh': 'cars_per_household', 
                                         'pst_mvp': 'postal_code_center'
                                        })

df_renamed.head()

,neighbourhood,municipality,citizens,females,a_00_14,a_15_24,a_25_44,a_45_64,births,households_with_children,not_married,rental_perc,percentage_working,low_edu,mid_edu,high_edu,avg_income,cars_per_household,postal_code_center
3,Appingedam-Centrum,Appingedam,2370,1230,230,225,500,715,10,250,985,59,61,480,650,720,.,"0,9",9901
4,Appingedam-West,Appingedam,3035,1530,480,350,570,990,25,480,1285,31,66,680,1150,520,.,"1,2",9903
5,Appingedam-Oost,Appingedam,5725,2960,900,580,1205,1560,40,865,2415,60,57,1460,2200,440,"24,2","0,9",9902
6,Verspreide huizen Damsterdiep en Eemskanaal,Appingedam,325,150,65,45,65,115,0,60,170,5,78,40,140,70,.,"1,8",9903
7,Verspreide huizen ten zuiden van Eemskanaal,Appingedam,100,45,10,15,10,45,0,15,40,12,.,40,30,10,.,.,9902


In [4]:
def percentage_missing(df = df_renamed, missing_value = '.', percentage = True):
    ''' Counts missing values as specified in missing_value.
    Optionally percentage can be set to False for the count of values
    '''
    total = df.shape[0]
    for column in df.columns:
        count = df[df[column] == missing_value].shape[0]
        if count != 0:
            if percentage:
                print(f'{column}: {round(count/total*100)}%')
            else:
                print(f'{column}: {count}')

percentage_missing(df_renamed, '.', percentage = True)

rental_perc: 11%
percentage_working: 26%
low_edu: 14%
mid_edu: 14%
high_edu: 14%
avg_income: 97%
cars_per_household: 21%
postal_code_center: 1%


In [5]:
for column in df_renamed.columns:
    if (not column  in ('neighbourhood','municipality')):
        df_renamed[column] = df_renamed[column].replace('.', 0).apply(str).str.replace(',', '.').astype(float)

In [6]:
from sklearn.preprocessing import MinMaxScaler

def demand(df_renamed = df_renamed, 
           citizens_weight = 1, females_weight = 1, a_00_14_weight = 1, a_45_64_weight = 1, births_weight = 1, households_with_children_weight = 1,
           singles_weight = 1, rental_weight = 1, working_weight = 1, low_edu_weight = 1, high_edu_weight = 1, cars_weight = 1,
               
           citizens_quantile = 0.5, females_quantile = 0.5, a_00_14_quantile = 0.5, a_45_64_quantile = 0.5, births_quantile = 0.5, households_with_children_quantile = 0.5,
           singles_quantile = 0.5, rental_quantile = 0.5, working_quantile = 0.5, low_edu_quantile = 0.5, high_edu_quantile = 0.5, cars_quantile = 0.5):
    ''' When a variable is not to be considered the weight should be set to zero
    For higher weight, >1  and lower 0 < weight < 1
    
    Quantiles set the demand for each variable. When higher, only higher values will be taken into account for demand calculation (and vice versa).
    '''
    df = df_renamed.copy()
    # Children under 15
    df['demand_a_00_14']= df['a_00_14']\
        .apply(lambda x: a_00_14_weight*1 if (x > df['a_00_14'].quantile(a_00_14_quantile)) else 0 )
    # + 45 < Age < 64
    df['demand_a_45_64']= df['a_45_64']\
        .apply(lambda x: a_45_64_weight*1 if (x > df['a_45_64'].quantile(a_45_64_quantile)) else 0 )
    # Households with children
    df['demand_households_with_children']= df['households_with_children']\
        .apply(lambda x: households_with_children_weight*1 if (x > df['households_with_children'].quantile(households_with_children_quantile)) else 0 )
    # cars per household
    df['demand_cars']= df['cars_per_household']\
        .apply(lambda x: cars_weight*1 if (x > df['cars_per_household'].quantile(cars_quantile)) else 0 )
    # births
    df['demand_births']= df['births']\
        .apply(lambda x: births_weight*1 if (x > df['births'].quantile(births_quantile)) else 0 )
    # rental homes
    df['demand_rental']= df['rental_perc']\
        .apply(lambda x: rental_weight*1 if (x > df['rental_perc'].quantile(rental_quantile)) else 0 )
    # %female
    df['demand_females']= df['females']\
        .apply(lambda x: females_weight*1 if (x > df['females'].quantile(females_quantile)) else 0 )
    # %singles
    df['demand_singles']= df['not_married']\
        .apply(lambda x: singles_weight*1 if (x > df['not_married'].quantile(singles_quantile)) else 0 )
    # education level
    df['demand_low_edu']= df['low_edu']\
        .apply(lambda x: low_edu_weight*1 if (x > df['low_edu'].quantile(low_edu_quantile)) else 0 )
    df['demand_high_edu']= df['high_edu']\
        .apply(lambda x: high_edu_weight*1 if (x > df['high_edu'].quantile(high_edu_quantile)) else 0 )
    # %unemployed
    df['demand_percentage_working']= df['percentage_working']\
        .apply(lambda x: working_weight*1 if (x > df['percentage_working'].quantile(working_quantile)) else 0 )
    # town size
    df['demand_citizens']= df['citizens']\
        .apply(lambda x: citizens_weight*1 if (x > df['citizens'].quantile(citizens_quantile)) else 0 )
    return df

if from_scratch:
    df_demand = demand(df_renamed)

# Playground Demand
Playground demand = (% of children <15) 
'+ (households with children (can be a percentage)) 
'+ (amount of births) 
'+ (income of households) 
'- (cars per household) 
'+ (% rental homes) 


In [7]:
def demand_play(df):
    ''' When a variable is not to be considered the weight should be set to zero
    For higher weight, >1  and lower <1
    '''
    df['play_demand'] = df['demand_a_00_14']\
                          + df['demand_households_with_children']\
                          + df['demand_cars']\
                          + df['demand_births']\
                          + df['demand_rental']
    df['play_demand'] = MinMaxScaler().fit_transform(np.array(df['play_demand']).reshape(-1,1))
    df['play_demand'] = df['play_demand'].apply(lambda x: round(x,3))
 
    return df

if from_scratch:
    df_demand = demand_play(df_demand)

## Sports
Sport_demand = - age - presence of children + (%male) + (%singles — rather than married) + household income + education level + (%unemployed) + town size


In [8]:
def demand_sport(df):
    ''' When a variable is not to be considered the weight should be set to zero
    For higher weight, >1  and lower <1
    '''
    df['sport_demand'] = df['demand_a_00_14']\
                          - df['demand_a_45_64']\
                          - df['demand_females']\
                          + df['demand_singles']\
                          - df['demand_households_with_children']\
                          - df['demand_low_edu']\
                          + df['demand_high_edu']\
                          - df['demand_percentage_working']\
                          + df['demand_citizens']
    df['sport_demand'] = MinMaxScaler().fit_transform(np.array(df['sport_demand']).reshape(-1,1))
    df['sport_demand'] = df['sport_demand'].apply(lambda x: round(x,3))
     
    return df

if from_scratch:
    df_demand = demand_sport(df_demand)

## Parks and Nature
Family with children (+), low-income (-), middle- to high income (+), low-education (-), high-education (+), 0-16 (++), 16-55 (+), >55 (-), female (+), single (+), distance > 1000m (+) 


In [9]:
def demand_park(df):
    # calculated sum of demand
    df['park_demand'] = df['demand_households_with_children']\
                          - df['demand_low_edu']\
                          + df['demand_high_edu']\
                          + df['demand_a_00_14']\
                          - df['demand_a_45_64']\
                          + df['demand_females']\
                          + df['demand_singles']
    df['park_demand'] = MinMaxScaler().fit_transform(np.array(df['park_demand']).reshape(-1,1))
    df['park_demand'] = df['park_demand'].apply(lambda x: round(x,3)) 
    return df

if from_scratch:
    df_demand = demand_park(df_demand)

In [10]:
def Demand_with_distance(df_demand, minimals = minimals, play_distance_min = 0.4, park_distance_min = 1, sport_distance_min = 1):
    df = df_demand.merge(minimals, left_on='neighbourhood', right_on='neighbourhood')

    for index, row in df.iterrows():
        if (row['play_distance'] <= play_distance_min):
            df.loc[index, 'play_demand'] = 0.0

        if (row['park_distance'] <= park_distance_min):
            df.loc[index, 'park_demand'] = 0.0

        if (row['sport_distance'] <= sport_distance_min):
            df.loc[index, 'sport_demand'] = 0.0
            
    return df

if from_scratch:
    df_demand = Demand_with_distance(df_demand, minimals)

In [11]:
if from_scratch:
    # Importing coordinates
    post_coordinate_map = pd.read_csv('postcode_coordinate.csv')
    post_coordinate_map = post_coordinate_map[['postcode', 'provincie', 'latitude', 'longitude']]

    # This drops 74 rows
    cbs_data_prepped = df_demand[df_demand['postal_code_center'] != '.' ]
    cbs_data_prepped['postal_code_center'] = cbs_data_prepped['postal_code_center'].astype(int)

    with_coordinates = pd.merge(cbs_data_prepped, post_coordinate_map,  how='left', left_on=['postal_code_center'], right_on = ['postcode'])
    with_coordinates.drop(columns=['postcode'], inplace=True)
    
    # Save to csv
    with_coordinates.to_csv('with_coordinates.csv')
    
else:
    # Importing minimal distance
    with_coordinates = pd.read_csv('with_coordinates.csv')
    with_coordinates = with_coordinates.drop(columns='Unnamed: 0')

#with_coordinates.head()

In [12]:
def Demand_threshold(with_coordinates = with_coordinates, play_demand_threshold = 0.7, park_demand_threshold = 0.7, sport_demand_threshold = 0.6):
    # Playground
    play_output_list = with_coordinates[['neighbourhood','municipality', 'a_00_14', 'births', 'rental_perc','play_demand', 'latitude', 'longitude','play_distance']]
    # Filter by play_demand_threshold
    play_output_list = play_output_list[play_output_list['play_demand'] > play_demand_threshold] 
    play_output_list = play_output_list.sort_values(by=['play_demand'], ascending=False)
    # Save
    play_output_list.to_csv('play_output_list.csv', index=False)

    # Park
    park_output_list = with_coordinates[['neighbourhood','municipality', 'households_with_children', 'high_edu', 'a_00_14', 'a_45_64', 'females', 'not_married', 'park_demand', 'latitude', 'longitude', 'park_distance']]
    park_output_list = park_output_list[park_output_list['park_demand'] > park_demand_threshold] 
    park_output_list = park_output_list.sort_values(by=['park_demand'], ascending=False)
    park_output_list.to_csv('park_output_list.csv', index=False)

    # Sport
    sport_output_list = with_coordinates[['neighbourhood','municipality', 'a_00_14', 'a_15_24', 'a_25_44', 'a_45_64', 'sport_demand', 'latitude', 'longitude', 'sport_distance']]
    sport_output_list = sport_output_list[sport_output_list['sport_demand'] > sport_demand_threshold] 
    sport_output_list = sport_output_list.sort_values(by=['sport_demand'], ascending=False)
    sport_output_list.to_csv('sport_output_list.csv', index=False)
    
    return play_output_list, park_output_list, sport_output_list

if from_scratch:
    play_output_list, park_output_list, sport_output_list = Demand_threshold(with_coordinates, play_demand_threshold = 0.7, park_demand_threshold = 0.7, sport_demand_threshold = 0.6)
        
else:
    # Importing minimal distance
    play_output_list = pd.read_csv('play_output_list.csv')
    park_output_list = pd.read_csv('park_output_list.csv')
    sport_output_list = pd.read_csv('sport_output_list.csv')

## Dash

In [13]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, dash_table, State
import dash_bootstrap_components as dbc
import plotly
import plotly.express as px

# Data
neighbourhoods = with_coordinates['neighbourhood'].unique()
municipalities = with_coordinates['municipality'].unique()
municipalities = np.append(municipalities,'Netherlands')

    
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='homepage')
])

df = with_coordinates.copy()
### Index page ###

index_page = html.Div([
    html.H1('Blind Spot Map for playgrounds, parks and sports facilities'),
    dcc.Link('Go to the Blind Spot Map', href='/map'),
    html.Br(),
    dcc.Link('Go to the Neighbourhood Charts', href='/chart'),
    dcc.Markdown('''
Blind Spot formulas:
* Playground demand = (% of children <15) + (Households with children) + (amount of births) - (cars per household) + (% rental homes)
* Sport_demand = - (age) - (Households with children) - (% female) + (%singles) + (education level) + (%unemployed) + (town size)
* Park demand = - (age) + (Households with children) + (% female) + (%singles) + (education level)

Datasets: 
* Centraal Bureau voor de Statistiek, subject to the [Creative Commons Naamsvermelding (CC BY 4.0)](https://www.cbs.nl/nl-nl/over-ons/website/copyright)
* Open Street Map, licensed under the [Open Data Commons Open Database License (ODbL)](https://www.openstreetmap.org/copyright) by the OpenStreetMap Foundation (OSMF)'''),
])



### Page 1 ###
page_1_layout = html.Div([
    html.H1('Blind Spot Map'),
    html.Div([
        # Map
        html.Div([
            html.B('Municipality', style={'font-size': '18px'}),
            dcc.Dropdown(municipalities, value='Netherlands', id='dropdown_province'),
            dcc.RadioItems(id='radio', options = ['Playgrounds', 'Sports Facilities','Parks & Nature'], value='Playgrounds',
                           inline=True, inputStyle={"margin-left": "20px"}),
            dcc.Graph(id='map'),
        ], style={'width': '65%', 'float': 'left', 'display': 'inline-block'}),
    
        # Manual inputs    
        html.Div([
            html.Div([
                html.Div([
                    html.B('Minimum Demand Value', style={'font-size': '16px'}),
                    dcc.Slider(min=0, max=1, value=0.7, id='slider_threshold'),
                ], style={'width': '48%', 'float': 'left'}),
                html.Div([
                    html.B('Minimum distance to facilities', style={'font-size': '16px', 'text-align':'right'}),
                    dcc.Input(value=1, type='number', id='input_distance'),
                ], style={'width': '48%', 'float': 'right'})
            ], className='row'),
            
            html.Br(),
            
            html.Div([
                 html.H2('Quantile'),
                 html.Label('Citizens quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='citizens_quantile'),
                 html.Label('Females quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='females_quantile'),
                 html.Label('A 00 14 quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='a_00_14_quantile'),
                 html.Label('A 45 64 quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='a_45_64_quantile'),
                 html.Label('Births quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='births_quantile'),
                 html.Label('Households with children quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='households_with_children_quantile'),
                 html.Label('Singles quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='singles_quantile'),
                 html.Label('Rental quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='rental_quantile'),
                 html.Label('Working quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='working_quantile'),
                 html.Label('Low edu quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='low_edu_quantile'),
                 html.Label('High edu quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='high_edu_quantile'),
                 html.Label('Cars quantile'),
                 dcc.Slider(min=0, max=1, value=0.5, id='cars_quantile'),
            ], className='row', style={'width': '50%', 'float': 'left', 'display': 'inline-block'}),
            
            html.Div([
                 html.H2('Weight'),
                 html.Label('Citizens weight'),
                 dcc.Input(value=1, type='number', id='citizens_weight'),
                 html.Label('Females weight'),
                 dcc.Input(value=1, type='number', id='females_weight'),
                 html.Label('A 00 14 weight'),
                 dcc.Input(value=1, type='number', id='a_00_14_weight'),
                 html.Label('A 45 64 weight'),
                 dcc.Input(value=1, type='number', id='a_45_64_weight'),
                 html.Label('Births weight'),
                 dcc.Input(value=1, type='number', id='births_weight'),
                 html.Label('Households with children weight'),
                 dcc.Input(value=1, type='number', id='households_with_children_weight'),
                 html.Label('Single weight'),
                 dcc.Input(value=1, type='number', id='singles_weight'),
                 html.Label('Rental weight'),
                 dcc.Input(value=1, type='number', id='rental_weight'),
                 html.Label('Working weight'),
                 dcc.Input(value=1, type='number', id='working_weight'),
                 html.Label('Low edu weight'),
                 dcc.Input(value=1, type='number', id='low_edu_weight'),
                 html.Label('High edu weight'),
                 dcc.Input(value=1, type='number', id='high_edu_weight'),
                 html.Label('Cars weight'),
                 dcc.Input(value=1, type='number', id='cars_weight'),
            ], className='row', style={'width': '50%', 'float': 'right', 'display': 'inline-block'}),
            
            html.Br(),
            html.Hr(),
            
            html.Div([
                html.A(html.Button('Reset values'),href='/', style={'width': '50%', 'float': 'left'}),
                html.Button('Download', id='button_download', n_clicks=0, style={'width': '50%', 'float': 'right'}),
                dcc.Download(id="download-dataframe-xlsx"),
            ]),
            
        ], className='row', style={'width': '25%', 'float': 'right', 'display': 'inline-block'}),
        
    ], className='row'),
    
    # Links
    html.Br(),
    dcc.Link('Go to Neighbourhood charts', href='/chart'),
    html.Br(),
    dcc.Link('Go back to the homepage', href='/'),
    
    # Down Table
    html.Div([
        dbc.Container(id='tbl_out')
    ], className='row'),
    
      
])

@app.callback(
    Output("map", "figure"),
    Output('tbl_out', 'children'),
    Input("radio", "value"), 
    Input('dropdown_province', 'value'),
    Input('slider_threshold', 'value'),   
    Input('input_distance', 'value'), 
    # 12 weights
    Input('citizens_weight', 'value'),
    Input('females_weight', 'value'),
    Input('a_00_14_weight', 'value'),
    Input('a_45_64_weight', 'value'),
    Input('births_weight', 'value'),
    Input('households_with_children_weight', 'value'),
    Input('singles_weight', 'value'),
    Input('rental_weight', 'value'),
    Input('working_weight', 'value'),
    Input('low_edu_weight', 'value'),
    Input('high_edu_weight', 'value'),
    Input('cars_weight', 'value'),
    # 12 Quantiles
    Input('citizens_quantile', 'value'),
    Input('females_quantile', 'value'),
    Input('a_00_14_quantile', 'value'),
    Input('a_45_64_quantile', 'value'),
    Input('births_quantile', 'value'),
    Input('households_with_children_quantile', 'value'),
    Input('singles_quantile', 'value'),
    Input('rental_quantile', 'value'),
    Input('working_quantile', 'value'),
    Input('low_edu_quantile', 'value'),
    Input('high_edu_quantile', 'value'),
    Input('cars_quantile', 'value'),  
)
def update_map(radio_input, municipality, slider_threshold, input_distance,
               citizens_weight, females_weight, a_00_14_weight, a_45_64_weight, births_weight, households_with_children_weight,
               singles_weight, rental_weight, working_weight, low_edu_weight, high_edu_weight, cars_weight,
               
               citizens_quantile, females_quantile, a_00_14_quantile, a_45_64_quantile, births_quantile, households_with_children_quantile,
               singles_quantile, rental_quantile, working_quantile, low_edu_quantile, high_edu_quantile, cars_quantile):
    
    df = with_coordinates.copy()
        
        
    # Table            
    if municipality != 'Netherlands':
        zoom = 11
        df = df[df['municipality'] == municipality]
        df_table = df.loc[:,['neighbourhood', 'citizens', 'females', 'a_00_14', 'a_15_24', 'a_25_44', 'a_45_64', 
        'births', 'households_with_children', 'not_married', 'rental_perc', 'percentage_working', 'low_edu', 'mid_edu', 'high_edu', 'cars_per_household',
        'play_demand', 'sport_demand', 'park_demand', 'sport_distance', 'play_distance', 'park_distance']]
        if(radio_input == "Playgrounds"):
            df_table.sort_values(by='play_demand', ascending = False, inplace=True)
        if(radio_input == "Parks & Nature"):
            df_table.sort_values(by='park_demand', ascending = False, inplace=True)
        if(radio_input == "Sports Facilities"):
            df_table.sort_values(by='sport_demand', ascending = False, inplace=True)
        
        table_out = [dbc.Label(municipality), dash_table.DataTable(df_table.to_dict('records'),[{"name": i, "id": i} for i in df_table.columns], id='tbl')] 
        
        
    if municipality == 'Netherlands':
        clickData = None 
        zoom = 7
        table_out = []
    
        
    # Weights and Quantiles
    
    # Children under 15
    if (a_00_14_weight!=1 or a_00_14_quantile!=0.5):
        df['demand_a_00_14']= df['a_00_14']\
        .apply(lambda x: a_00_14_weight*1 if (x > df['a_00_14'].quantile(a_00_14_quantile)) else 0 )
    # + 45 < Age < 64
    if (a_45_64_weight!=1 or a_45_64_quantile!=0.5):
        df['demand_a_45_64']= df['a_45_64']\
        .apply(lambda x: a_45_64_weight*1 if (x > df['a_45_64'].quantile(a_45_64_quantile)) else 0 )
    # Households with children
    if (households_with_children_weight!=1 or households_with_children_quantile!=0.5):
        df['demand_households_with_children']= df['households_with_children']\
        .apply(lambda x: households_with_children_weight*1 if (x > df['households_with_children'].quantile(households_with_children_quantile)) else 0 )
    # cars per household
    if (cars_weight!=1 or cars_quantile!=0.5):
        df['demand_cars']= df['cars_per_household']\
        .apply(lambda x: cars_weight*1 if (x > df['cars_per_household'].quantile(cars_quantile)) else 0 )
    # births
    if (births_weight!=1 or births_quantile!=0.5):
        df['demand_births']= df['births']\
        .apply(lambda x: births_weight*1 if (x > df['births'].quantile(births_quantile)) else 0 )
    # rental homes
    if (rental_weight!=1 or rental_quantile!=0.5):
        df['demand_rental']= df['rental_perc']\
        .apply(lambda x: rental_weight*1 if (x > df['rental_perc'].quantile(rental_quantile)) else 0 )
    # %female
    if (females_weight!=1 or females_quantile!=0.5):
        df['demand_females']= df['females']\
        .apply(lambda x: females_weight*1 if (x > df['females'].quantile(females_quantile)) else 0 )
    # %singles
    if (singles_weight!=1 or singles_quantile!=0.5):
        df['demand_singles']= df['not_married']\
        .apply(lambda x: singles_weight*1 if (x > df['not_married'].quantile(singles_quantile)) else 0 )
    # + education level
    if (low_edu_weight!=1 or low_edu_quantile!=0.5):
        df['demand_low_edu']= df['low_edu']\
        .apply(lambda x: low_edu_weight*1 if (x > df['low_edu'].quantile(low_edu_quantile)) else 0 )
    if (high_edu_weight!=1 or high_edu_quantile!=0.5):
        df['demand_high_edu']= df['high_edu']\
        .apply(lambda x: high_edu_weight*1 if (x > df['high_edu'].quantile(high_edu_quantile)) else 0 )
    # + %unemployed
    if (working_weight!=1 or working_quantile!=0.5):
        df['demand_percentage_working']= df['percentage_working']\
        .apply(lambda x: working_weight*1 if (x > df['percentage_working'].quantile(working_quantile)) else 0 )
    # + town size
    if (citizens_weight!=1 or citizens_quantile!=0.5):
        df['demand_citizens']= df['citizens']\
        .apply(lambda x: citizens_weight*1 if (x > df['citizens'].quantile(citizens_quantile)) else 0 )

        
    # Calculating demands
    if(radio_input == "Playgrounds"):        
        # Calculating demand
        df['play_demand'] = df['demand_a_00_14']\
                          + df['demand_households_with_children']\
                          + df['demand_cars']\
                          + df['demand_births']\
                          + df['demand_rental']
        df['play_demand'] = MinMaxScaler().fit_transform(np.array(df['play_demand']).reshape(-1,1))
        df['play_demand'] = df['play_demand'].apply(lambda x: round(x,3))
        
        # Adding distance calculations
        for index, row in df.iterrows():
            if (row['play_distance'] <= input_distance):
                df.loc[index, 'play_demand'] = 0.0    
                
        # Filtering Demand threshold
        df = df[df['play_demand'] > slider_threshold] 
        
        # Map config: Steven's psychophysical law
        hover_data = ["play_demand", "play_distance"]
        size = df['play_demand'].apply(lambda x: x**(1/0.7))
        

    elif (radio_input == "Parks & Nature"):       
        # Calculating demand
        df['park_demand'] = df['demand_households_with_children']\
                      - df['demand_low_edu']\
                      + df['demand_high_edu']\
                      + df['demand_a_00_14']\
                      - df['demand_a_45_64']\
                      + df['demand_females']\
                      + df['demand_singles']
        df['park_demand'] = MinMaxScaler().fit_transform(np.array(df['park_demand']).reshape(-1,1))
        df['park_demand'] = df['park_demand'].apply(lambda x: round(x,3))
        
        # Adding distance calculations
        for index, row in df.iterrows():
            if (row['park_distance'] <= input_distance):
                df.loc[index, 'park_demand'] = 0.0
                
        # Filtering Demand threshold
        df = df[df['park_demand'] > slider_threshold]
        
        # Map config
        hover_data = ["park_demand", "park_demand"]
        size = df['park_demand'].apply(lambda x: x**(1/0.7))

    if(radio_input == "Sports Facilities"):       
        # Calculating demand
        df['sport_demand'] = df['demand_a_00_14']\
                          - df['demand_a_45_64']\
                          - df['demand_females']\
                          + df['demand_singles']\
                          - df['demand_households_with_children']\
                          - df['demand_low_edu']\
                          + df['demand_high_edu']\
                          - df['demand_percentage_working']\
                          + df['demand_citizens']
        df['sport_demand'] = MinMaxScaler().fit_transform(np.array(df['sport_demand']).reshape(-1,1))
        df['sport_demand'] = df['sport_demand'].apply(lambda x: round(x,3))
        
        # Adding distance calculations
        for index, row in df.iterrows():
            if (row['sport_distance'] <= input_distance):
                df.loc[index, 'sport_demand'] = 0.0
        
        # Filtering Demand threshold
        df = df[df['sport_demand'] > slider_threshold]
        
        # Map config
        hover_data = ["sport_demand", "sport_demand"]
        size = df['sport_demand'].apply(lambda x: x**(1/0.7))
    
    # Map
    hover_name = 'neighbourhood'
    fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", mapbox_style = "open-street-map", height = 900, opacity = 0.9, color_continuous_scale = plotly.colors.sequential.Purp,
                            color=size, size=size, zoom = zoom,  hover_name = hover_name, hover_data = hover_data)
    fig.update_layout(clickmode='event+select')
    
    return fig, table_out

# Click on map
@app.callback(
    Output('dropdown_province', 'value'),
    Input('map', 'clickData'))   
def display_click_data(clickData):
    neighbourhood_df = df[['neighbourhood', 'municipality']]
    if clickData != None:
        neighbourhood = dict(clickData['points'][0])['hovertext']
        municipality = list(neighbourhood_df[neighbourhood_df['neighbourhood'] == neighbourhood]['municipality'])[0]
    else:
        municipality = 'Netherlands'
    return municipality

# Download button
@app.callback(
    Output("download-dataframe-xlsx", "data"),
    Input("button_download", "n_clicks"),
    State('tbl', 'data'),
    State('tbl', 'columns'),
    State('dropdown_province', 'value'),
    prevent_initial_call=True,
)
def download(n_clicks, rows, columns, name):
    to_download = pd.DataFrame(rows, columns=[c['name'] for c in columns])
    return dcc.send_data_frame(to_download.to_excel, "Regio: "+ name + " blindspots.xlsx", sheet_name="Blindspots")




### Page 2 ###

page_2_layout = html.Div([
    html.H1('Neighbourhood demographics'),
    html.Div([
        html.B('Neighbourhood', style={'font-size': '18px'}),
        dcc.Dropdown(neighbourhoods, value='Kanaleneiland', id='dropdown_province'),
        dcc.Graph(id='chart_age'),
        dcc.Graph(id='chart_education'),
    ], className='row'),
    
    html.Div([
        html.Br(),
        dcc.Link('Go to the Blind Spot Map', href='/map'),
        html.Br(),
        dcc.Link('Go back to the homepage', href='/')
    ], className='row'),  
])
@app.callback(Output('chart_age','figure'),
              Output('chart_education','figure'),
              Input('dropdown_province', 'value'),
             )
def make_charts(neighbourhood):
    ### Age ###
    aggregations = {
    'citizens':'sum',
    'a_00_14':'sum',
    'a_15_24': 'sum',
    'a_25_44':'sum',
    'a_45_64': 'sum'
    }
    
    # Data
    df = with_coordinates.copy()
    # Neighbourhood
    df = df.loc[:,['neighbourhood', 'municipality', 'citizens', 'a_00_14', 'a_15_24', 'a_25_44', 'a_45_64']]
    # Municipality
    df_municipalities = df.groupby(['municipality'], as_index=False).agg(aggregations)

    # Country
    df_NL = df_municipalities.agg(aggregations)
    columns = df_NL.to_frame().index
    df_NL = pd.DataFrame([list(df_municipalities.agg(aggregations))],columns=columns)
    df_NL.index=['NL']
    
    municipality = list(df.loc[df['neighbourhood'] == neighbourhood,'municipality'])[0]
    df_neighbourhood = df.loc[df['neighbourhood']==neighbourhood]
    df_neighbourhood = df_neighbourhood.drop(columns=['municipality'])
    df_neighbourhood.set_index('neighbourhood', inplace=True)
    df_municipality = df_municipalities.loc[df_municipalities['municipality']==municipality]
    df_municipality.set_index('municipality', inplace=True)
    df_municipality

    graph_df = pd.concat([df_neighbourhood,df_municipality,df_NL])
    graph_df.loc[:,'a_65+'] = graph_df.loc[:,'citizens'] - graph_df.loc[:,'a_00_14'] - graph_df.loc[:,'a_15_24'] - graph_df.loc[:,'a_25_44'] - graph_df.loc[:,'a_45_64']
    graph_df = graph_df.T
    graph_df = round(graph_df.div(graph_df.iloc[0])*100,0)
    graph_df = graph_df.drop(['citizens'])

    chart_age = px.line(graph_df, x=graph_df.index, y=[neighbourhood,municipality,'NL'], title='Age', labels ={'index':'Age band','value':'Percentage(%)','variable':'Location'})
    
    
    
    ### Education ###
    aggregations = {
    'citizens':'sum',
    'low_edu': 'sum',
    'mid_edu':'sum',
    'high_edu': 'sum',
    }

    df = with_coordinates.copy()
    # Neighbourhood
    df = df.loc[:,['neighbourhood', 'municipality', 'citizens', 'low_edu', 'mid_edu', 'high_edu']]
    # Municipality
    df_municipalities = df.groupby(['municipality'], as_index=False).agg(aggregations)

    # Country
    df_NL = df_municipalities.agg(aggregations)
    columns = df_NL.to_frame().index
    df_NL = pd.DataFrame([list(df_municipalities.agg(aggregations))],columns=columns)
    df_NL.index=['NL']


    # Making the df
    df_neighbourhood = df.loc[df['neighbourhood']==neighbourhood]
    df_neighbourhood = df_neighbourhood.drop(columns=['municipality'])
    df_neighbourhood.set_index('neighbourhood', inplace=True)
    df_municipality = df_municipalities.loc[df_municipalities['municipality']==municipality]
    df_municipality.set_index('municipality', inplace=True)
    df_municipality

    graph_df = pd.concat([df_neighbourhood,df_municipality,df_NL])
    graph_df.loc[:,'Others'] = graph_df.loc[:,'citizens'] - graph_df.loc[:,'low_edu'] - graph_df.loc[:,'mid_edu'] - graph_df.loc[:,'high_edu']
    graph_df = graph_df.T
    graph_df = round(graph_df.div(graph_df.iloc[0])*100,0)
    graph_df = graph_df.drop(['citizens'])

    chart_education = px.line(graph_df, x=graph_df.index, y=[neighbourhood,municipality,'NL'], title='Education Level', labels ={'index':'Education level','value':'Percentage(%)','variable':'Location'})
    
    return chart_age, chart_education

# Update the index
@app.callback(Output('homepage', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/map':
        return page_1_layout
    elif pathname == '/chart':
        return page_2_layout
    else: # 404 "URL not found"
        return index_page
    
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/


In [14]:
attributes = ['citizens', 'females', 'a_00_14', 'a_15_24', 'a_25_44', 'a_45_64', 'births', 'households_with_children',
'not_married', 'rental_perc', 'percentage_working', 'low_edu', 'mid_edu', 'high_edu', 'cars_per_household',]

Weight = ['citizens_weight', 'females_weight', 'a_00_14_weight', 'a_45_64_weight', 'births_weight', 'households_with_children_weight', 'singles_weight', 'rental_weight',
               'working_weight', 'low_edu_weight', 'high_edu_weight', 'cars_weight'] 
    
Quantile = ['citizens_quantile', 'females_quantile', 'a_00_14_quantile', 'a_45_64_quantile', 'births_quantile', 'households_with_children_quantile', 'singles_quantile',
'rental_quantile', 'working_quantile',  'low_edu_quantile', 'high_edu_quantile', 'cars_quantile']
         

all_options = {
    'Weight': Weight,
    'Quantile': Quantile
}

# radio_output_quantile
radio_output_quantile = []
for i in all_options['Quantile']:
    radio_output_quantile.append(
    html.Label(i.replace('_',' ').capitalize()))
    radio_output_quantile.append(
    dcc.Slider(
    id=i,
    min=0,
    max=1,
    value = 0.5))
    
# radio_output_weight
radio_output_weight = []
for i in all_options['Weight']:
    radio_output_weight.append(
    html.Label(i.replace('_',' ').capitalize()))
    radio_output_weight.append(dcc.Input(id=i, value=1, type='number'))